### AutoGen Example Tool Use: Get Current time based on city

In [62]:
import os
import dotenv
from autogen import ConversableAgent
import openai

dotenv.load_dotenv("..\\common\\credentials.env", verbose=True, override=True)

True

Default config for autogen points to open ai format APIs. 
To use azure API's set config as below

In [63]:
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
aoai_api_type = os.environ["OPENAI_API_TYPE"]
deployment_name = "gpt-4o" # "gpt-4" #use GPT-4 as 3.5 turbo seems to not work well for math problems
aoai_api_version = "2024-02-15-preview" #Using latest version as of this date. Older versions may not work with the code as syntax has changed drastically.

config_list = [
    {
        "model": deployment_name,  
        "base_url": aoai_endpoint, 
        "api_type": aoai_api_type, 
        "api_version": aoai_api_version, 
        "api_key": aoai_api_key,
        "temperature": 0.7
 }
]

In [64]:
# importing datetime
from datetime import datetime

# importing pytz module
import pytz



def getCurrentTime(city: str) -> str:
    # giving the format of datetime
    format = "%Y-%m-%d %H:%M:%S %Z%z"

    # getting the standard UTC time
    original_tz = pytz.utc 

    if city == "Mumbai":
        converted_tz = pytz.timezone('Asia/Kolkata')
    elif city == "New York":
        converted_tz = pytz.timezone('US/Eastern')
    elif city == "Dallas":
        converted_tz = pytz.timezone('US/Central')
    elif city == "Los Angeles":
        converted_tz = pytz.timezone('US/Pacific')
    else:
        converted_tz = original_tz

    return datetime.now(converted_tz).strftime(format)

  

In [65]:
getCurrentTime("New York")

'2024-12-18 19:12:00 EST-0500'

In [ ]:
import os

from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
time_agent = ConversableAgent(
    name="TimeAgent",
    system_message="""You are a helpful AI assistant who help answering questions about current time based on the city.
    If question is not about time then dont suggest function call instead reply based on your knowledge and add keyword 'TERMINATE' at the end. 
    Return 'TERMINATE' when the task is done.""",
    llm_config={"config_list": config_list},
)

# The proxy_agent is used for  function calling and executing code
#Also, it will be used for interacting with the time agent.
proxy_agent = ConversableAgent(
    name="ProxyAgent",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER"
)



<function __main__.getCurrentTime(city: str) -> str>

In [71]:
from autogen import register_function


# Register the tool signature with the time assistant agent.
time_agent.register_for_llm(name="getCurrentTime", description="Gives you current time based on city")(getCurrentTime)

# Register the tool function with the proxy agent.
proxy_agent.register_for_execution(name="getCurrentTime")(getCurrentTime)

# Register the getCurrentTime function to the two agents.
register_function(
    getCurrentTime,
    caller=time_agent,  # The TimeAgent can suggest call to the getCurrentTime function.
    executor=proxy_agent,  # The code executor agent can execute the function.    
    name="getCurrentTime",  # By default, the function name is used as the tool name.
    description="Gives you current time based on city",  # A description of the tool.
)

Initiate chat using user proxy(executor) agent

In [72]:
chat_result = proxy_agent.initiate_chat(time_agent, message="What is current time in Dallas")

ProxyAgent (to TimeAgent):

What is current time in Dallas

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
TimeAgent (to ProxyAgent):

***** Suggested tool call (call_LGoYczDvWxt17hZnVMfTJTn0): getCurrentTime *****
Arguments: 
{"city":"Dallas"}
*******************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION getCurrentTime...
ProxyAgent (to TimeAgent):

ProxyAgent (to TimeAgent):

***** Response from calling tool (call_LGoYczDvWxt17hZnVMfTJTn0) *****
2024-12-18 18:32:55 CST-0600
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
TimeAgent (to ProxyAgent):

The current time in Dallas is 6:32 PM CST on December 18, 2024. TERMINATE

----------------------------------------

In [73]:
chat_result = proxy_agent.initiate_chat(time_agent, message="who is president of united states")

ProxyAgent (to TimeAgent):

who is president of united states

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
TimeAgent (to ProxyAgent):

As of October 2023, the President of the United States is Joe Biden. TERMINATE

--------------------------------------------------------------------------------
